In [1]:
from bedrock_agentcore_starter_toolkit.operations.memory.manager import Memory, MemoryManager

In [2]:
print("\n" + "="*25 + " CONTROL PLANE DEMO " + "="*25)

manager = MemoryManager(region_name="us-east-1")

print("🔍 DEBUG: Starting control plane operations...")

memory: Memory = manager.get_or_create_memory(
    name="DemoLongTermMemory",
    description="A temporary memory for short-lived conversations."
)
print("🔍 DEBUG: Short-term memory created successfully")
memory

✅ MemoryManager initialized for region: us-east-1



========================= CONTROL PLANE DEMO =========================
🔍 DEBUG: Starting control plane operations...


Memory already exists. Using existing memory ID: DemoLongTermMemory-9t9zeCFzmr
🔎 Retrieving memory resource with ID: DemoLongTermMemory-9t9zeCFzmr...
  ✅ Found memory: DemoLongTermMemory-9t9zeCFzmr


🔍 DEBUG: Short-term memory created successfully


{'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory-9t9zeCFzmr', 'id': 'DemoLongTermMemory-9t9zeCFzmr', 'name': 'DemoLongTermMemory', 'description': 'A memory for long-lived conversations.', 'eventExpiryDuration': 90, 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 654000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 840000, tzinfo=tzlocal()), 'strategies': [{'strategyId': 'fact_extractor-srMyD87TfT', 'name': 'fact_extractor', 'description': 'Extracts and stores factual information', 'type': 'SEMANTIC', 'namespaces': ['support/user/{sessionId}/{actorId}/facts'], 'createdAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 654000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 840000, tzinfo=tzlocal()), 'status': 'ACTIVE'}]}

In [3]:
# MEMORY_ID = "DemoLongTermMemory-uJZDq62oUz"

In [4]:
# List all memories
for memory_summary in manager.list_memories():
    print(f"🔍 DEBUG: Memory found : {memory_summary}")

🔍 DEBUG: Memory found : {'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory-9t9zeCFzmr', 'id': 'DemoLongTermMemory-9t9zeCFzmr', 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 654000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 840000, tzinfo=tzlocal()), 'memoryId': 'DemoLongTermMemory-9t9zeCFzmr'}


In [5]:
# Get all the memory strategies available
strategies = manager.get_memory_strategies(memory_id=memory.id)

In [6]:
if "SummaryStrategy" not in [stragegy.name for stragegy in strategies]:
    manager.add_summary_strategy_and_wait(
        memory_id=memory.id,
        name="SummaryStrategy",
        namespaces=["support/user/{actorId}/{sessionId}"],
    )
    print("🔍 DEBUG: Summary strategy added successfully")
else:
    print("🔍 DEBUG: Summary strategy already exists - skipping update")

Updated memory strategies for: DemoLongTermMemory-9t9zeCFzmr
Waiting for memory DemoLongTermMemory-9t9zeCFzmr to return to ACTIVE state and strategies to reach terminal states...
Memory DemoLongTermMemory-9t9zeCFzmr is ACTIVE and all strategies are in terminal states (took 0 seconds)


🔍 DEBUG: Summary strategy added successfully


In [7]:
# Using direct access capability to show memory fields
memory.description

'A memory for long-lived conversations.'

In [8]:
# Fetch the memory again to see the updated strategies
get_response = manager.get_memory(memory_id=memory.id)

🔎 Retrieving memory resource with ID: DemoLongTermMemory-9t9zeCFzmr...
  ✅ Found memory: DemoLongTermMemory-9t9zeCFzmr


In [9]:
get_response.status

'ACTIVE'

In [10]:
strategy = manager.get_memory_strategies(memory_id=memory.id)[0]
strategy

{'strategyId': 'SummaryStrategy-MyxOdo9AGm', 'name': 'SummaryStrategy', 'type': 'SUMMARIZATION', 'namespaces': ['support/user/{actorId}/{sessionId}'], 'createdAt': datetime.datetime(2025, 9, 22, 17, 28, 15, 587000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 22, 17, 28, 15, 587000, tzinfo=tzlocal()), 'status': 'ACTIVE'}

In [11]:
manager.delete_strategy(memory_id=memory.id, strategy_id=strategy["strategyId"])
manager.list_memories()

Updated memory strategies for: DemoLongTermMemory-9t9zeCFzmr


[{'arn': 'arn:aws:bedrock-agentcore:us-east-1:328307993871:memory/DemoLongTermMemory-9t9zeCFzmr', 'id': 'DemoLongTermMemory-9t9zeCFzmr', 'status': 'ACTIVE', 'createdAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 654000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 22, 17, 28, 16, 359000, tzinfo=tzlocal()), 'memoryId': 'DemoLongTermMemory-9t9zeCFzmr'}]

In [12]:
manager.get_memory_strategies(memory_id=memory.id)

[{'strategyId': 'fact_extractor-srMyD87TfT', 'name': 'fact_extractor', 'description': 'Extracts and stores factual information', 'type': 'SEMANTIC', 'namespaces': ['support/user/{sessionId}/{actorId}/facts'], 'createdAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 654000, tzinfo=tzlocal()), 'updatedAt': datetime.datetime(2025, 9, 18, 19, 3, 35, 840000, tzinfo=tzlocal()), 'status': 'ACTIVE'}]

In [13]:
# try:
#     manager.delete_memory(memory_id=memory.id)
# except Exception as e:
#     print(f"🔍 DEBUG: Error deleting memory: {e}")
#     pass

In [14]:
try:
    memory = manager.get_memory(memory_id=memory.id)
    print(f"🔍 DEBUG: Memory found with status: {memory.status}")
except Exception as e:
    print(f"🔍 DEBUG: Memory deletion confirmed. Error: {e}")

🔎 Retrieving memory resource with ID: DemoLongTermMemory-9t9zeCFzmr...
  ✅ Found memory: DemoLongTermMemory-9t9zeCFzmr


🔍 DEBUG: Memory found with status: ACTIVE
